In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) 

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [4]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[7 6 5 ... 7 2 2] [2 4 4 ... 1 7 8]
0.09329268292682927
Iteration:  10
[2 3 4 ... 2 1 5] [2 4 4 ... 1 7 8]
0.21409756097560975
Iteration:  20
[2 3 4 ... 1 1 5] [2 4 4 ... 1 7 8]
0.2868292682926829
Iteration:  30
[2 3 4 ... 1 7 5] [2 4 4 ... 1 7 8]
0.369390243902439
Iteration:  40
[2 4 4 ... 1 7 4] [2 4 4 ... 1 7 8]
0.43041463414634146
Iteration:  50
[2 4 4 ... 1 7 4] [2 4 4 ... 1 7 8]
0.47729268292682925
Iteration:  60
[2 4 4 ... 1 7 8] [2 4 4 ... 1 7 8]
0.518609756097561
Iteration:  70
[2 4 4 ... 1 7 8] [2 4 4 ... 1 7 8]
0.554439024390244
Iteration:  80
[2 4 4 ... 1 7 8] [2 4 4 ... 1 7 8]
0.5832195121951219
Iteration:  90
[2 4 4 ... 1 7 8] [2 4 4 ... 1 7 8]
0.6094878048780488
Iteration:  100
[2 4 4 ... 1 7 8] [2 4 4 ... 1 7 8]
0.6318048780487805
Iteration:  110
[2 4 4 ... 1 7 8] [2 4 4 ... 1 7 8]
0.6522926829268293
Iteration:  120
[2 4 4 ... 1 7 8] [2 4 4 ... 1 7 8]
0.6693414634146342
Iteration:  130
[2 4 4 ... 1 7 8] [2 4 4 ... 1 7 8]
0.6853414634146342
Iteration:  140


In [7]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [8]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[4 8 3 1 2 6 3 4 0 9 4 5 9 5 8 4 5 6 1 2 6 3 8 1 2 9 4 7 4 7 2 0 0 9 1 9 6
 3 4 9 6 2 0 4 9 6 8 4 0 5 9 3 3 8 0 9 2 2 1 1 4 3 9 4 7 1 4 6 4 1 3 5 7 4
 2 7 3 0 7 0 5 7 3 0 3 9 2 4 9 8 0 0 9 8 6 6 8 0 7 1 8 3 2 3 6 0 8 1 6 8 2
 5 5 8 1 1 9 9 4 7 8 6 6 1 9 4 4 4 5 0 1 5 1 1 5 6 2 8 4 4 2 5 7 4 8 9 2 4
 7 6 0 6 2 2 0 4 9 7 2 4 5 7 4 3 3 8 7 0 9 4 9 8 9 9 4 7 0 3 8 8 8 0 1 6 4
 5 0 6 0 0 3 9 4 9 3 0 6 6 2 2 1 8 8 4 8 0 1 2 4 9 2 4 2 1 6 5 3 9 6 2 0 8
 2 8 3 3 6 3 0 8 8 6 2 7 7 8 7 1 6 7 9 8 0 7 0 4 5 1 5 5 8 0 7 0 8 0 8 7 6
 1 3 6 7 0 9 3 6 9 4 7 2 6 5 2 2 7 1 6 8 0 1 1 8 0 8 8 6 6 6 3 0 3 1 1 0 6
 5 7 0 1 8 9 6 7 7 8 6 8 6 3 3 6 2 8 4 3 3 2 1 1 3 2 8 4 9 1 1 6 3 6 0 9 7
 6 7 4 5 9 8 3 1 1 0 8 4 3 9 4 1 8 8 3 9 3 3 6 1 6 5 7 7 2 8 0 6 7 5 0 2 1
 1 3 3 6 0 5 4 6 1 2 2 3 8 6 4 8 7 7 8 7 3 7 3 1 6 2 5 6 3 9 6 9 4 1 0 2 7
 0 2 5 8 5 9 6 9 5 9 9 8 4 8 8 0 7 4 7 1 9 4 5 1 3 9 9 0 0 4 5 2 7 8 8 0 5
 0 3 5 5 4 7 4 5 4 1 5 1 6 2 5 3 5 4 3 6 9 5 7 5 5 5 7 6 1 1 6 6 1 1 4 9 9
 4 1 8 3 1 6 2 7 9 1 6 5 

0.827